<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Define-dataframes-and-function" data-toc-modified-id="Define-dataframes-and-function-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Define dataframes and function</a></span><ul class="toc-item"><li><span><a href="#Read-in-data" data-toc-modified-id="Read-in-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Read in data</a></span></li><li><span><a href="#Combine-pieces-from-parallel-runs" data-toc-modified-id="Combine-pieces-from-parallel-runs-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Combine pieces from parallel runs</a></span></li><li><span><a href="#print_row_detail-function" data-toc-modified-id="print_row_detail-function-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>print_row_detail function</a></span></li></ul></li><li><span><a href="#Test-and-refine-regex-patterns-for-flagging-likely-relevant-documents" data-toc-modified-id="Test-and-refine-regex-patterns-for-flagging-likely-relevant-documents-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Test and refine regex patterns for flagging likely relevant documents</a></span><ul class="toc-item"><li><span><a href="#function:-check_regex_match" data-toc-modified-id="function:-check_regex_match-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>function: check_regex_match</a></span></li><li><span><a href="#Test-cases-for-regex" data-toc-modified-id="Test-cases-for-regex-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Test cases for regex</a></span></li><li><span><a href="#Working-list-for-flagging-positive-matches" data-toc-modified-id="Working-list-for-flagging-positive-matches-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Working list for flagging positive matches</a></span></li><li><span><a href="#Assign-initial-positive-and-negative-labels-using-regex" data-toc-modified-id="Assign-initial-positive-and-negative-labels-using-regex-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Assign initial positive and negative labels using regex</a></span><ul class="toc-item"><li><span><a href="#function:-label_with_regex" data-toc-modified-id="function:-label_with_regex-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>function: label_with_regex</a></span></li></ul></li><li><span><a href="#Create-lists-for-exploring-results-(to-be-replaced-by-using-DF)" data-toc-modified-id="Create-lists-for-exploring-results-(to-be-replaced-by-using-DF)-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Create lists for exploring results (to be replaced by using DF)</a></span></li><li><span><a href="#function:-print_docs_from_list" data-toc-modified-id="function:-print_docs_from_list-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>function: print_docs_from_list</a></span></li><li><span><a href="#Patterns-to-identify-paragraphs-with-few-or-no-numerals" data-toc-modified-id="Patterns-to-identify-paragraphs-with-few-or-no-numerals-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Patterns to identify paragraphs with few or no numerals</a></span></li><li><span><a href="#Check-for-false-positives" data-toc-modified-id="Check-for-false-positives-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Check for false positives</a></span></li></ul></li><li><span><a href="#Write-current-work-to-csv" data-toc-modified-id="Write-current-work-to-csv-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Write current work to csv</a></span></li></ul></div>

# Label paragraphs

This notebook labels paragraphs as "relevant" (positive hits) or not. 
Relevant = contains the employee count for the entire company for the most recent year. 

To do:
- Create reusable regex units, then compose regex using string join
- Modify print_row_detail function so that it can return the row index
    - Needed in order to create a list of false positives or negatives to correct

In [1]:
import numpy as np
import pandas as pd
import re

from bs4 import BeautifulSoup as bs

from path import Path, getcwdu

import glob
import os
from pathlib import PurePath
import copy

import random
import gzip
import shutil

## Define dataframes and function

### Read in data

In [2]:
full_path_list = [PurePath(os.getcwd()).joinpath(file).as_posix() for file in glob.iglob('../employee_filings/*.gz')]
full_file_list = [PurePath(file).name for file in glob.iglob('../employee_filings/*.gz')]
full_accession_ids = [PurePath(file).stem.replace('.html', '') for file in full_file_list]
full_cik_nbrs = [x.split(sep='-')[0] for x in full_accession_ids]

In [3]:
train_accession_ids = pd.read_csv('../data/train_accession_ids.csv', names=['acc_id'])['acc_id'].tolist()
val_accession_ids = pd.read_csv('../data/val_accession_ids.csv', names=['acc_id'])['acc_id'].tolist()

labeled_df = pd.read_excel('../data/train_val_employee_count_paragraphs.xlsx')
subset_df = pd.read_excel('../data/subset_employee_count_paragraphs.xlsx')

In [4]:
subset_file_list = [PurePath(os.getcwd()).joinpath('../employee_filings/').joinpath(file) for file in full_file_list if PurePath(file).stem.replace('.html', '') in subset_df.accession_number.unique().tolist()]

In [67]:
# After reading back in, the html needs to be parsed again
#tbl_html_df = pd.read_csv('data/tbl_html_df.csv')
tbl_html_df = pd.read_csv('data/tbl_html_df3.csv')
tbl_html_df['tbl_html'] = tbl_html_df.tbl_html.apply(lambda x: bs(x, 'lxml'))

In [61]:
p_dtypes = {'acc_id' : 'category', 
'len' : 'int', 
'emp_header' : 'bool', 
'first_emp_head_block' : 'bool', 
'split' : 'category', 
'label' : 'uint8'}
#paragraph_input_df = pd.read_csv('data/paragraph_input_df.csv', dtype=p_dtypes, index_col=0)
#paragraph_input_df = pd.read_csv('data/paragraph_input_df2.csv', dtype=p_dtypes, index_col=0)
paragraph_input_df = pd.read_csv('data/paragraph_input_df3.csv', dtype=p_dtypes, index_col=0)
train_df = paragraph_input_df[paragraph_input_df.split == 'train']

val_df = paragraph_input_df[paragraph_input_df.split == 'val']

### Combine pieces from parallel runs

In [56]:
"""
paragraph_input_df_1 = pd.read_csv('data/paragraph_input_df_1.csv', dtype=p_dtypes, index_col=0)
paragraph_input_df_2 = pd.read_csv('data/paragraph_input_df_2.csv', dtype=p_dtypes, index_col=0)
paragraph_input_df_3 = pd.read_csv('data/paragraph_input_df_3.csv', dtype=p_dtypes, index_col=0)
paragraph_input_df_4 = pd.read_csv('data/paragraph_input_df_4.csv', dtype=p_dtypes, index_col=0)

print(paragraph_input_df.shape)
print(paragraph_input_df_1.shape)
print(paragraph_input_df_2.shape)
print(paragraph_input_df_3.shape)
print(paragraph_input_df_4.shape)
"""

(9217, 9)
(3775, 10)
(4132, 10)
(4442, 10)
(3996, 10)


In [63]:
"""
tbl_html_df_1 = pd.read_csv('data/tbl_html_df_1.csv', dtype=p_dtypes, index_col=0)
tbl_html_df_2 = pd.read_csv('data/tbl_html_df_2.csv', dtype=p_dtypes, index_col=0)
tbl_html_df_3 = pd.read_csv('data/tbl_html_df_3.csv', dtype=p_dtypes, index_col=0)
tbl_html_df_4 = pd.read_csv('data/tbl_html_df_4.csv', dtype=p_dtypes, index_col=0)

print(tbl_html_df.shape)
print(tbl_html_df_1.shape)
print(tbl_html_df_2.shape)
print(tbl_html_df_3.shape)
print(tbl_html_df_4.shape)
"""

(208, 4)
(57, 4)
(57, 4)
(26, 4)
(81, 4)


In [64]:
"""
tbl_html_df_1['idx'] = tbl_html_df_1.index.values
tbl_html_df_2['idx'] = tbl_html_df_2.index.values + tbl_html_df_1.shape[0] 
tbl_html_df_3['idx'] = tbl_html_df_3.index.values + tbl_html_df_2.shape[0] + tbl_html_df_1.shape[0]
tbl_html_df_4['idx'] = tbl_html_df_4.index.values + tbl_html_df_3.shape[0] + tbl_html_df_2.shape[0] + tbl_html_df_1.shape[0]
print(tbl_html_df_1.head(1)['idx'])
print(tbl_html_df_1.tail(1)['idx'])
print(tbl_html_df_2.head(1)['idx'])
print(tbl_html_df_2.tail(1)['idx'])
print(tbl_html_df_3.head(1)['idx'])
print(tbl_html_df_3.tail(1)['idx'])
print(tbl_html_df_4.head(1)['idx'])
print(tbl_html_df_4.tail(1)['idx'])
"""

0    0
Name: idx, dtype: int64
56    56
Name: idx, dtype: int64
0    57
Name: idx, dtype: int64
56    113
Name: idx, dtype: int64
0    114
Name: idx, dtype: int64
25    139
Name: idx, dtype: int64
0    140
Name: idx, dtype: int64
80    220
Name: idx, dtype: int64


In [65]:
#tbl_html_df3 = pd.concat([tbl_html_df_1, tbl_html_df_2, tbl_html_df_3, tbl_html_df_4]).set_index('idx', drop=True)
#tbl_html_df3.to_csv('data/tbl_html_df3.csv')

In [57]:
"""
paragraph_input_df_2['idx'] = paragraph_input_df_2.index.values + paragraph_input_df_1.shape[0] 
paragraph_input_df_3['idx'] = paragraph_input_df_3.index.values + paragraph_input_df_2.shape[0] + paragraph_input_df_1.shape[0]
paragraph_input_df_4['idx'] = paragraph_input_df_4.index.values + paragraph_input_df_3.shape[0] + paragraph_input_df_2.shape[0] + paragraph_input_df_1.shape[0]
print(paragraph_input_df_1.head(1)['idx'])
print(paragraph_input_df_1.tail(1)['idx'])
print(paragraph_input_df_2.head(1)['idx'])
print(paragraph_input_df_2.tail(1)['idx'])
print(paragraph_input_df_3.head(1)['idx'])
print(paragraph_input_df_3.tail(1)['idx'])
print(paragraph_input_df_4.head(1)['idx'])
print(paragraph_input_df_4.tail(1)['idx'])
"""

0    0
Name: idx, dtype: int64
3774    3774
Name: idx, dtype: int64
0    3775
Name: idx, dtype: int64
4131    7906
Name: idx, dtype: int64
0    7907
Name: idx, dtype: int64
4441    12348
Name: idx, dtype: int64
0    12349
Name: idx, dtype: int64
3995    16344
Name: idx, dtype: int64


In [58]:
#paragraph_input_df3 = pd.concat([paragraph_input_df_1, paragraph_input_df_2, paragraph_input_df_3, paragraph_input_df_4]).set_index('idx', drop=True)
#paragraph_input_df3.to_csv('data/paragraph_input_df3.csv')

In [59]:
len(paragraph_input_df3.acc_id.unique())

2216

### print_row_detail function

In [52]:
def print_row_detail(df=train_df, nrow=10, header_list = ['acc_id'],
                    detail_list = [ 'len', 'first_emp_head_block', 'split' ,'para_text'],
                    sortby=['acc_id', 'first_emp_head_block', 'len'], ascending=False):
    df_sorted = df.sort_values(sortby, ascending=ascending).reset_index()
    nrow = min(len(df_sorted), nrow)
    for i in range(0, nrow):
        for h in header_list:
            print('-'*35  + ' ' +  str(df_sorted[h][i]) + ' ' + '-'*35)
        for d in detail_list:
            print(d + '  :' + str(df_sorted[d][i]))
            print('')

In [53]:
print_row_detail()

----------------------------------- 0001703056-18-000008 -----------------------------------
len  :241

first_emp_head_block  :True

split  :train

para_text  :As of December 31, 2017, we employed approximately 18,000 people. Approximately 10% of our employees are covered by collective bargaining agreements. We believe that our relations with our employees and labor unions have generally been good.

----------------------------------- 0001703056-18-000008 -----------------------------------
len  :1249

first_emp_head_block  :False

split  :train

para_text  :James D. DeVries is our President. From May 2, 2016 to September 2017, Mr. DeVries served as The ADT Corporation's Executive Vice President and Chief Operating Officer. From December 2014 to May 2016, Mr. DeVries served as Executive Vice President of Brand Operations at Allstate Insurance Company, the second largest personal lines insurer in the United States. During his tenure at Allstate, Mr. DeVries led the operations organizati

## Test and refine regex patterns for flagging likely relevant documents

### function: check_regex_match

In [54]:
def check_regex_match(pattern, text_list):
    for idx, s in enumerate(text_list):
        mo = re.search(pattern, s)
        if mo:
            ms = mo.span()[1]
            print("------    " + str(idx) + "   Matched!    -----")
            print('str length  :' + str(len(s)) + '    match span  :' + str(ms))
            print(s[:ms])
            print('')
            print(s[ms:])
            print(re.search(pattern, s))
        else:
            print("------    " + str(idx) + "  NO MATCH    -----")
            print(s)

### Test cases for regex

In [55]:
test_list = ["We are a small company with approximately 61 employees.",
"The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31, 2015. "
"Total workforce level at December 31, 2016 was approximately 150,500",           
"Currently, the Company and its subsidiaries have an aggregate of 35 employees.",
"we employ only 31 employees", 
"We currently have 21 employees",
"((we|the compnay) )(currently )?((has|have|employ(s)?) )(the equivalent of )([0-9]{1,3},)*[0-9]{1,3}?"
"We currently employ 26 full-time employees",
"Including our full and part-time personnel, we estimate that we have \
             the equivalent of 12 full time employees."
"As a REIT, we employ only 31 employees and have a cost-effective \
             management structure."
            ]

check_regex_match(re.compile(r"((number of|head[ -]?count|total|workforce) )[^.]+ (employees|team members) [^.]+ ([0-9]{1,3},)*[0-9]{1,3}[^\d]", re.I),
                 test_list)

------    0  NO MATCH    -----
We are a small company with approximately 61 employees.
------    1   Matched!    -----
str length  :196    match span  :121
The number of full-time employees of the Company was approximately 31,800 at December 31, 2016 and 32,300 at December 31,

 2015. Total workforce level at December 31, 2016 was approximately 150,500
<_sre.SRE_Match object; span=(4, 121), match='number of full-time employees of the Company was >
------    2  NO MATCH    -----
Currently, the Company and its subsidiaries have an aggregate of 35 employees.
------    3  NO MATCH    -----
we employ only 31 employees
------    4  NO MATCH    -----
We currently have 21 employees
------    5  NO MATCH    -----
((we|the compnay) )(currently )?((has|have|employ(s)?) )(the equivalent of )([0-9]{1,3},)*[0-9]{1,3}?We currently employ 26 full-time employees
------    6  NO MATCH    -----
Including our full and part-time personnel, we estimate that we have              the equivalent of 12 full tim

### Working list for flagging positive matches

In [140]:
employee_terms = "(associates|employees|full[ -]time[ -]equivalent(s)?|staff|team members|workers)"
person_terms = "(individuals|people|persons)"
workforce_terms = "(((employee|employment|head|personnel|staff|worker|workforce) (count(s)|level(s)|total(s))+)|(head-count|headcount|workforce))"
employee_type_terms = "(full time|full-time|permanent|part time|part-time|regular|seasonal|temporary|total)"

numeral_pat = "(([0-9]{1,3},)*[0-9]{1,3}([.][0-9])?)"
rel_qualifiers = "(a total of|approximately|in aggregate|in total|(an|the) equivalent of|total)"

space_pat = "( |\n)"

In [141]:
magnitude_words = "(hundred|thousand|million|billion)" 
num_words = "(one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen)"

In [142]:
num_pat = "".join(["((",numeral_pat, "|", num_words, ")(", space_pat, magnitude_words, ")?", ")"])

num_emps_pat = "".join([num_pat, space_pat, "(", rel_qualifiers, space_pat, ")*", 
                        "(", employee_type_terms, space_pat, ")*"])

In [143]:
number_employees_pat = "".join([num_emps_pat, employee_terms])
employed_num_pat = "".join(["employ((ed|s)?)?", space_pat, "(", rel_qualifiers, space_pat, ")*", 
                        num_emps_pat])
employed_words_num_pat = "".join(["employ((ed|s)?)?", "[^.]+ *", numeral_pat])
workforce_words_num_pat = "".join([workforce_terms, "[^.]+ *", numeral_pat])


emp_pat_list = [number_employees_pat, employed_num_pat, 
                employed_words_num_pat, workforce_words_num_pat, 
               r"((number of|head[ -]?count|total|workforce) )[^.]+ (employees|team members) [^.]+ ([0-9]{1,3},)*[0-9]{1,3}[^\d]", 
               r"(employees)(of the company )?((is|was|approximately) )*([0-9]{1,3},)*[0-9]{1,3}"]
emp_pats = [re.compile(x, re.I) for x in emp_pat_list]

In [105]:
flag_pat_list = [
r"employ((ed|s)?) ((approximately|the equivalent of) )*([0-9]{1,3},)*[0-9]{1,3} ((permanent|full|time|part|time|temporary|total|and|equivalent)[ -])*(employees|individuals|people|team members|members|persons|associates)",
r"([0-9]{1,3},)*[0-9]{1,3} [^.]+ ((employ\w*|head([ -])?count|member(s)?|person(s)?|people|staff|team|workforce))",
r"((employ(ed)?|employee count|head([ -])?count|member(s)?|person(s)?|people|staff|team|workforce) )+[^.]+ *([0-9]{1,3},)*[0-9]{1,3}",
r"((number of|head[ -]?count|total|workforce) )[^.]+ (employees|team members) [^.]+ ([0-9]{1,3},)*[0-9]{1,3}[^\d]",
r"([0-9]{1,3},)*[0-9]{1,3} ((permanent|full|time|part|time|temporary|total|and|equivalent)[ -])*(employees|team members)",
r"(employees)(of the company )?((is|was|approximately) )*([0-9]{1,3},)*[0-9]{1,3}"]


#r"((permanent|full|part|time|full-time|part-time|temporary) )+(employees|team members|associates)",
#r"employed( approximately)?$", 
#r"Total workforce",

#r"^((permanent|full|part|time|full-time|part-time|temporary|total) )*(employees|team members|associates)"
                 
flag_pats = [re.compile(x, re.I) for x in flag_pat_list]

In [48]:
para_list = train_df.para_text.tolist()

val_para_list = val_df.para_text.tolist()

### Assign initial positive and negative labels using regex

#### function: label_with_regex

In [70]:

def label_with_regex(df, regex_list, text_col = 'para_text', label_col = 'label', 
                    reset_labels_first = True):
    """Label df in place if text matches any regex in regex_list"""
    df_para_list = df[text_col].tolist()
    pos_ids = set()
    for i, para in enumerate(df_para_list):
    ## "any() does not work for this purpose - it takes some explosive logic path"
    #if not any(reg.search(para) for reg in nonum_regs) and k2 not in para_dict[k]['df'].keys():
        for reg in regex_list:
            if reg.search(para):
                pos_ids.add(df.index[i])
                break
    if reset_labels_first:
        df.loc[:, label_col] = 0
    df.loc[df.index.isin(pos_ids), label_col] = 1

In [145]:
label_with_regex(train_df, emp_pats)
label_with_regex(val_df, emp_pats)

C:\Anaconda\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### Create lists for exploring results (to be replaced by using DF)

In [75]:
train_df[np.logical_and(train_df.label ==0, 
                        train_df.len <= 21)][train_df.len >11].para_text.value_counts()

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Workers' compensation    38
Workers' Compensation    23
D. Employees             19
Workers compensation      9
Total employees           8
Workforce Reductions      7
Workforce Solutions       7
Total Employees           6
D.  Employees             6
Workforce reduction       5
Workforce reductions      5
Employees at year end     5
Capital employed          5
Workers Compensation      5
Workforce Reduction       4
Team Members              4
Staffing services         3
Stafford, VA              3
Stafford, TX              3
Total headcount           3
D. EMPLOYEES              3
Headcount Reduction       2
Total associates          2
Staff advances (a)        2
Total workforce           2
Employees (thousand)      2
Average headcount         2
Total staff costs         2
D. Employees.             2
WORKERS COMPENSATION      2
                         ..
Total team members        1
Total Associates          1
NATG - Workforce          1
Full-time equivalents     1
Stafford / SUSA Dr  

In [146]:
nonum_pat = re.compile(r"^([^\d]|[$][\d]+)[^\d]*$", re.I)
pos_key_list = train_df[(train_df.label ==1) & (train_df.len > 19)].acc_id.tolist()
pos_list = train_df[(train_df.label ==1) & (train_df.len > 19)].para_text.tolist()
neg_key_list = train_df[(train_df.label ==0) & (train_df.len > 19)].acc_id.tolist()
neg_list = train_df[np.logical_and(train_df.label ==0, train_df.len > 19)].para_text.tolist()
neg_key_list = [x for x in neg_key_list if not re.match(nonum_pat, neg_list[neg_key_list.index(x)])]
neg_list = [x for x in neg_list if not re.match(nonum_pat, x)]


In [156]:
nonum_pat = re.compile(r"^([^\d]|[$][\d]+)[^\d]*$", re.I)
pos_key_list_val = val_df[(val_df.label ==1) & (val_df.len > 19)].acc_id.tolist()
pos_list_val = val_df[(val_df.label ==1) & (val_df.len > 19)].para_text.tolist()
neg_key_list_val = val_df[(val_df.label ==0) & (val_df.len > 19)].acc_id.tolist()
neg_list_val = val_df[np.logical_and(val_df.label ==0, val_df.len > 19)].para_text.tolist()
neg_key_list_val = [x for x in neg_key_list_val if not re.match(nonum_pat, neg_list_val[neg_key_list_val.index(x)])]
neg_list_val = [x for x in neg_list_val if not re.match(nonum_pat, x)]


In [158]:
print('Total paragraphs: ' + str(len(pos_list + neg_list)))

print('Positive paragraphs: ' + str(len(pos_list)))
print('Total keys: ' + str(len(train_df.acc_id.unique())))
print('Keys with positive labels: ' + str(len(train_df[train_df.label ==1].acc_id.unique())))
print('Negative paragraphs: ' + str(len(neg_list)))
print('Keys with negative labels: ' + str(len(train_df[train_df.label ==0].acc_id.unique())))

Total paragraphs: 7801
Positive paragraphs: 5409
Total keys: 1661
Keys with positive labels: 1618
Negative paragraphs: 2392
Keys with negative labels: 1572


Validation labels

In [157]:
print('Total paragraphs: ' + str(len(pos_list_val + neg_list_val)))

print('Positive paragraphs: ' + str(len(pos_list_val)))
print('Total keys: ' + str(len(val_df.acc_id.unique())))
print('Keys with positive labels: ' + str(len(val_df[val_df.label ==1].acc_id.unique())))
print('Negative paragraphs: ' + str(len(neg_list_val)))
print('Keys with negative labels: ' + str(len(val_df[val_df.label ==0].acc_id.unique())))

Total paragraphs: 2676
Positive paragraphs: 1863
Total keys: 555
Keys with positive labels: 543
Negative paragraphs: 813
Keys with negative labels: 535


In [78]:
train_df[train_df.len == 10].head(20)

,acc_id,para_text,len,emp_header,first_emp_head_block,para_text_orig,para_tag,split,label
idx,,,,,,,,,
180,0000011544-17-000009,Employees,10,False,False,Employees,"<div class=""c240""><span class=""c239"">Employees...",train,0
278,0000018498-16-000065,Employees',10,False,False,Employees',"<div class=""c19"">Employees'</div>",train,0
280,0000018498-16-000065,Employees',10,False,False,Employees',"<div class=""c205"">Employees'</div>",train,0
281,0000018498-16-000065,Employees',10,False,False,Employees',"<div class=""c334"">Employees'</div>",train,0
1068,0000068709-17-000026,Employees,10,False,False,Employees,"<div class=""c1""><span class=""c76"">Employees</s...",train,0
1111,0000072207-17-000014,Employees,10,False,False,Employees,"<div class=""c172""><span class=""c56"">Employees<...",train,0
1372,0000089089-17-000006,Associates,10,False,False,Associates,"<div class=""c67"">Associates</div>",train,0
1671,0000105634-17-000043,Employees,10,False,False,Employees,"<div class=""c77""><span class=""c80""> </span><a ...",train,0
2184,0000717538-17-000060,Employees',10,False,False,Employees',"<div class=""c53"">Employees'</div>",train,0


### function: print_docs_from_list

In [79]:
def print_docs_from_list(key_list: list, doc_list: list, start: int=0, ndocs: int=50):
    end = start + ndocs
    for idx, tup in enumerate(zip(key_list[start:end], doc_list[start:end])):
        print(str(idx + start) + '   ------   ' + str(tup[0]))
        print(tup[1])

### Patterns to identify paragraphs with few or no numerals

These paragraphs are often not relevant

In [153]:
nonum_pats = [r"^([^\d]|[$][\d]+)[^\d]+$",
    r"^[^\d]{0,2}\d{1,2}[^\d]*$", 
    r"^[^\d]* \d{1,2}[^\d]*$", 
    r"^([^\d]+(\d{1,2}[^\d]{2,}){1,5}\d{1,3}[^\d]*)$",
    r"^((\d{1,3}[^\d]{2,}){1,4}\d{1,2}[^\d]*)$"]
year_and_num = re.compile(r"20[0-2][0-9].*[0-9]{1,3}.*|[0-9]{1,3}.*20[0-2][0-9]", re.I)
nonum_regs = [re.compile(x) for x in nonum_pats]

In [154]:
for y in nonum_regs:
    print(len([x for x in pos_list if re.search(y, x)]))

71
7
79
94
3


In [150]:
for y in nonum_regs:
    print(len([x for x in neg_list if re.search(y, x)]))

0
23
114
80
2


In [155]:
check_pat = nonum_regs[0]
check_regex_match(check_pat, [x for x in pos_list if re.search(check_pat, x)])

------    0   Matched!    -----
str length  :1074    match span  :1074
In determining the fair market value of securities held in the Trust Fund which are listed on a registered stock exchange, the Administrator shall direct the Trustee to value the same at the prices they were last traded on such exchange preceding the close of business on the Valuation Date. If such securities were not traded on the Valuation Date, or if the exchange on which they are traded was not open for business on the Valuation Date, then the securities shall be valued at the prices at which they were last traded prior to the Valuation Date. Any unlisted security held in the Trust Fund shall be valued at its bid price next preceding the close of business on the Valuation Date, which bid price shall be obtained from a registered broker or an investment banker. In determining the fair market value of assets other than securities for which trading or bid prices can be obtained, the Trustee may appraise such assets

### Check for false positives

In [84]:
row_filter = np.logical_and(train_df.label == 1, 
    train_df.para_text.apply(lambda x: bool(re.search(nonum_pats[3], x)))
                           )

print_row_detail(df=train_df[row_filter], nrow=50, header_list = ['acc_id'],
                    detail_list = [ 'len', 'first_emp_head_block','para_text'],
                    sortby=['first_emp_head_block','acc_id',  'len'], ascending=False)

----------------------------------- 0001672764-17-000046 -----------------------------------
len  :203

first_emp_head_block  :False

para_text  :Currently our board has 11 members (see "Item 6. Directors, Senior Management and Employees-A. Directors and Senior Management-Board of Directors"). The board meets as often as required by our interests.

----------------------------------- 0001648416-17-000016 -----------------------------------
len  :1349

first_emp_head_block  :False

para_text  :Dutch legal requirements concerning director independence differ in certain respects from the rules applicable to U.S. companies listed on the NYSE. While under most circumstances both regimes require that a majority of board members be "independent," the definition of this term under Dutch law differs from the definition used under the NYSE corporate governance standards. In some cases the Dutch requirement is more stringent, such as by requiring a longer "look- back" period (five years) for form

This code will probably be replaced by the dataframe version, but the dataframe version needs to return the index first

In [60]:
print_docs_from_list(doc_list=[x for x in pos_list if re.search(nonum_pats[2], x)], 
    key_list=[pos_key_list[i] for i in [pos_list.index(x) for x in pos_list if re.search(nonum_pats[2], x)] ])

0   ------   0000014846-16-000062
We share facilities, personnel and other resources with several affiliated entities including, among others, Gould Investors L.P., a master limited partnership involved primarily in the ownership and operation of a diversified portfolio of real estate assets, and One Liberty Properties, Inc., an NYSE listed equity REIT. Eight individuals (including Jeffrey A. Gould, Chief Executive Officer and President, Mitchell Gould, Executive Vice President and George Zweier, Chief Financial Officer), devote substantially all of their business time to our activities, while our other personnel (including several officers) share their services on a part-time basis with us and other affiliated entities that share our executive offices. (Including our full and part-time personnel, we estimate that we have the equivalent of 12 full time employees. The allocation of expenses for the shared facilities, personnel and other resources is computed in accordance with a Shared 

## Write current work to csv

This allows code cleanup in this notebook while tuning classifiers in another notebook

In [152]:
train_df.to_csv('data/classifier_input_train3.csv')
val_df.to_csv('data/classifier_input_val3.csv')